# Reading in the files

In [15]:
import cv2
import numpy as np

#Read the image, convert it into grayscale, and make in binary image for threshold value of 1.
img = cv2.imread('frame.png')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
_,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)

#Now find contours in it. There will be only one object, so find bounding rectangle for it.

contours = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnt = contours[0]
x,y,w,h = cv2.boundingRect(cnt)

#Now crop the image, and save it into another file.

crop = img[y:y+h,x:x+w]
cv2.imwrite('framemod.png',crop)

True

In [26]:
import numpy as np

def autocrop(image, threshold=0):
    """Crops any edges below or equal to threshold

    Crops blank image to 1x1.

    Returns cropped image.

    """
    if len(image.shape) == 3:
        flatImage = np.max(image, 2)
    else:
        flatImage = image
    assert len(flatImage.shape) == 2

    rows = np.where(np.max(flatImage, 0) > threshold)[0]
    if rows.size:
        cols = np.where(np.max(flatImage, 1) > threshold)[0]
        image = image[cols[0]: cols[-1] + 1, rows[0]: rows[-1] + 1]
    else:
        image = image[:1, :1]

    return image

img = cv2.imread('frame.png')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
crop = autocrop(gray)
cv2.imwrite('framemodOther.png',crop)



True

In [23]:
def crop_image(img,tol=0):
    # img is image data
    # tol  is tolerance
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

img = cv2.imread('frame.png')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
crop = autocrop(gray)
cv2.imwrite('framemodThird.png',crop)

True

## Image histogram equalization

In [40]:
from skimage import data, img_as_float
from skimage import exposure
import cv2
import numpy as np

img1 = cv2.imread('histogramNormalization/first.png',0)
img2 = cv2.imread('histogramNormalization/second.png',0)
# Equalization
#img = cv2.imread('wiki.jpg',0)
equ1 = cv2.equalizeHist(img1)
equ2 = cv2.equalizeHist(img2)
res = np.hstack((equ1,equ2)) #stacking images side-by-side
cv2.imwrite('histogramNormalization/res.png',res)


# create a CLAHE object (Arguments are optional).
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
cl1 = clahe.apply(img1)
cl2 = clahe.apply(img2)
res_cl = np.hstack((cl1, cl2)) #stacking images side-by-side
cv2.imwrite('histogramNormalization/res_cl.png',res_cl)



True

# Itertools combinations

In [11]:
from itertools import combinations

# key value pair (key, value)

example_hash_strings= [("0WS86GPURFK5","c68686868e0f0e1c"), 
                    ("76KUS3QCGVCY","c78786868e0f0e1c") ,
                    ("96EC4QS20Z28","c78786868e0f0e1c"),
                    ("CL8W7L333U90","c78706868e0f0e1c"),
                    ("FDAZ5NL5NFL2","c7870646ce0f0e1c"),
                    ("HBX8QLI9HH25","c7870686ce0f0e1c"),
                    ("JY2ZAINWD2RX","c68706068e0e0e1c"),
                    ("LP47ZGJ256YU","c78786068e0f0e1e"),
                    ("NTETO8P77N96","c78786868e0f0e1c"),
                    ("SLK2PRXGW3DZ","c78706868e0f0e1c")]

example_hash_strings2= [("0WS86GPURFK5","c68686868e0f0e1c"), 
                    ("76KUS3QCGVCY","c78786868e0f0e1c") ]

# find all combinations
for i in combinations(example_hash_strings2, 2):
    # calculate difference in hash for each pair
    print i



(('0WS86GPURFK5', 'c68686868e0f0e1c'), ('76KUS3QCGVCY', 'c78786868e0f0e1c'))


'0WS86GPURFK5'